In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load your dataset
df_raw = pd.read_csv('vgsales_consolidated.csv')  # Replace with your actual dataset

# Select relevant columns
df_raw = df_raw[['Name', 'Genre', 'Publisher', 'Year', 'Global_Sales', 'Decade', 'FranchiseTag']].dropna()
df_raw = df_raw.reset_index(drop=True)

# summary of df columns
print(df_raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11338 entries, 0 to 11337
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          11338 non-null  object 
 1   Genre         11338 non-null  object 
 2   Publisher     11338 non-null  object 
 3   Year          11338 non-null  int64  
 4   Global_Sales  11338 non-null  float64
 5   Decade        11338 non-null  int64  
 6   FranchiseTag  11338 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 620.2+ KB
None


In [2]:
# Getting unique counts of categorical columns
print("Unique Genres:", df_raw['Genre'].nunique())
print("Unique Publishers:", df_raw['Publisher'].nunique())

Unique Genres: 12
Unique Publishers: 564


In [3]:
# One-hot encode categorical features
categorical_cols = ['Genre', 'Publisher']
df = pd.get_dummies(df_raw, columns=categorical_cols, drop_first=False)

# Scale numerical features
scaler = StandardScaler()
num_features = scaler.fit_transform(df[['Year']])

In [4]:
df.head()

,Name,Year,Global_Sales,Decade,FranchiseTag,Genre_action,Genre_adventure,Genre_fighting,Genre_misc,Genre_platform,...,Publisher_xseed games,Publisher_yacht club games,Publisher_yamasa entertainment,Publisher_yeti,Publisher_yukes,Publisher_yumedia,Publisher_zenrin,Publisher_zoo digital publishing,Publisher_zoo games,Publisher_zushi games
0,007 quantum of solace,2008,3.92,2000,0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,007 racing,2000,0.53,2000,0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,007 the world is not enough,2000,2.47,2000,0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,007 tomorrow never dies,1999,3.21,1990,0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1 vs 100,2008,0.09,2000,0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [44]:
from transformers import BertTokenizer, BertModel
import torch

# Load BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize titles
titles = df['Name'].tolist()
inputs = tokenizer(titles, padding=True, truncation=True, return_tensors="pt")

# Generate embeddings
with torch.no_grad():
    outputs = bert_model(**inputs)

# Use [CLS] token embedding
title_embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # shape: (num_samples, 768)


C:\Users\maxac\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [45]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

# assume title_embeddings is a numpy array shape (N, D) dtype float32
X = title_embeddings.astype(np.float32)  # (N, D)
ids = [f"doc_{i}" for i in range(X.shape[0])]
meta = df['Name'].astype(str).tolist()    # or other metadata

# Build pandas DataFrame with embedding as Python list per row
df_parquet = pd.DataFrame({
    "id": ids,
    "title": meta,
    "embedding": X.tolist()   # list[float]
})

# Create explicit pyarrow schema using list<float32>
pa_schema = pa.schema([
    pa.field("id", pa.string()),
    pa.field("title", pa.string()),
    pa.field("embedding", pa.list_(pa.float32()))
])

table = pa.Table.from_pandas(df_parquet, schema=pa_schema, preserve_index=False)
pq.write_table(table, "title_embeddings.parquet", compression="zstd", row_group_size=100_000)

In [5]:
import pyarrow.dataset as ds
import numpy as np

dataset = ds.dataset("title_embeddings.parquet", format="parquet")

for batch in dataset.to_batches(batch_size=50_000, columns=["id","title","embedding"]):
    pdf = batch.to_pandas()
    emb_batch = np.vstack(pdf["embedding"].values).astype(np.float32)  # (B, D)
    # use emb_batch for indexing or inference

In [6]:
# extracting column from pandas DataFrame
ids = pdf['id'].tolist()
titles = pdf['title']
embeddings = np.vstack(pdf['embedding'].values).astype(np.float32)  # (N, D)

In [7]:
embeddings.shape  # (N, D)

(11338, 768)

In [8]:
import numpy as np

# Combine all features
X = np.concatenate([
    titles.values.reshape(-1, 1),
    embeddings,
    df[['FranchiseTag']].values,
    df.iloc[:, 5:581].values,  # Columns 5-580, which are one-hot encoded categorical features
    num_features
], axis=1)

y = df['Global_Sales'].values
print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

Feature matrix shape: (11338, 1347)
Target vector shape: (11338,)


In [9]:
# Perform 80-20 train-test split
from sklearn.model_selection import train_test_split

X_train_name, X_test_name, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train_name[:, 1:].astype(np.float32)  # Exclude title column for model training
X_test = X_test_name[:, 1:].astype(np.float32)    # Exclude title column for model testing

print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (9070, 1346) (9070,)
Test set shape: (2268, 1346) (2268,)


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Hyperparameter tuning for KNN

param_grid = {
    'n_neighbors': range(1, 31),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p': [1, 2]
}

grid = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5)
grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
print("Best score:", grid.best_score_)


Best parameters: {'metric': 'euclidean', 'n_neighbors': 12, 'p': 1, 'weights': 'distance'}
Best score: 0.23877160965183145


In [15]:
knn_params = {'metric': 'euclidean', 'n_neighbors': 12, 'p': 1, 'weights': 'distance'}

In [12]:
# Evaluate on test set
best_knn = grid.best_estimator_
knn_pred = best_knn.predict(X_test)

# Constructing dataframe of names, global sales, and predictions
knn_results_df = pd.DataFrame({
    "Name": X_test_name[:, 0],  # Names from the name-included test set
    "Actual_Global_Sales": y_test,
    "Predicted_Global_Sales": knn_pred.flatten()
})

# viewing model predictions
knn_results_df

,Name,Actual_Global_Sales,Predicted_Global_Sales
0,chronicles of mystery the secret tree of life,0.08,0.252238
1,back at the barnyard slop bucket games,0.14,0.362859
2,earth defense force 2025,0.51,0.263754
3,scoobydoo unmasked,0.75,0.523497
4,mlb 10 the show,1.32,1.142959
...,...,...,...
2263,ben 10 omniverse 2,0.03,0.728105
2264,medabots metabee,0.05,0.539219
2265,katekyoo hitman reborn kindan no yami no delta,0.03,0.035720
2266,gaia saver hero saidai no sakusen,0.03,0.167483


In [13]:
# Calculating MAE
from sklearn.metrics import mean_absolute_error

knn_mae = mean_absolute_error(y_test, knn_pred)
knn_mae

0.7229640869951588

In [84]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='linear')  # Regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50


c:\Program Files\Python313\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 5.2596 - mae: 0.8827 - val_loss: 3.8917 - val_mae: 0.7821
Epoch 2/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.8218 - mae: 0.8497 - val_loss: 3.8096 - val_mae: 0.8496
Epoch 3/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.6246 - mae: 0.8111 - val_loss: 3.7559 - val_mae: 0.7107
Epoch 4/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.2125 - mae: 0.7908 - val_loss: 3.5595 - val_mae: 0.7279
Epoch 5/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.1036 - mae: 0.7697 - val_loss: 3.4643 - val_mae: 0.7039
Epoch 6/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.9316 - mae: 0.7751 - val_loss: 3.6013 - val_mae: 0.7442
Epoch 7/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.4664 - mae: 0.7622 - val_loss: 3.7546 - val_mae: 0.6827
Epoch 8/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.0080 - mae: 0.7299 - val_loss: 3.4884 - val_mae: 0.6784
Epoch 9/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.7529 

In [97]:
predictions = model.predict(X_test)

# Constructing dataframe of names, global sales, and predictions
results_df = pd.DataFrame({
    "Name": X_test_name[:, 0],  # Names from the name-included test set
    "Actual_Global_Sales": y_test,
    "Predicted_Global_Sales": predictions.flatten()
})

# viewing model predictions
results_df

71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


,Name,Actual_Global_Sales,Predicted_Global_Sales
0,chronicles of mystery the secret tree of life,0.08,0.229535
1,back at the barnyard slop bucket games,0.14,0.442490
2,earth defense force 2025,0.51,0.455672
3,scoobydoo unmasked,0.75,0.571154
4,mlb 10 the show,1.32,0.885334
...,...,...,...
2263,ben 10 omniverse 2,0.03,0.571189
2264,medabots metabee,0.05,0.354599
2265,katekyoo hitman reborn kindan no yami no delta,0.03,0.113134
2266,gaia saver hero saidai no sakusen,0.03,0.307830


In [98]:
# Displaying top 10 highest global sales out of the test set
top_10 = results_df.sort_values(by="Actual_Global_Sales", ascending=False).head(10)
top_10

,Name,Actual_Global_Sales,Predicted_Global_Sales
1852,pokemon redpokemon blue,31.37,9.170599
743,new super mario bros wii,28.62,3.075672
1480,call of duty black ops 3,25.32,24.753408
982,nintendogs,24.76,14.524964
97,wii fit,22.72,12.643062
175,call of duty advanced warfare,21.90,15.839271
1576,fifa 16,16.44,8.810226
575,fifa soccer 13,16.16,16.264153
1395,call of duty world at war,15.87,1.626569
1970,pokemon blackpokemon white,15.32,6.481796


In [99]:
# Displaying top 10 predicted sales out of the test set
top_10_pred = results_df.sort_values(by="Predicted_Global_Sales", ascending=False).head(10)
top_10_pred

,Name,Actual_Global_Sales,Predicted_Global_Sales
1480,call of duty black ops 3,25.32,24.753408
575,fifa soccer 13,16.16,16.264153
175,call of duty advanced warfare,21.90,15.839271
982,nintendogs,24.76,14.524964
1108,grand theft auto 2,3.42,13.551766
97,wii fit,22.72,12.643062
1660,assassins creed iii,13.10,10.782689
1867,call of duty modern warfare trilogy,0.04,10.420180
1643,the sims 4,2.97,10.139952
603,fifa soccer 07,6.38,9.998370


In [100]:
# Displaying bottom 10 predicted sales out of the test set
bottom_10_pred = results_df.sort_values(by="Predicted_Global_Sales", ascending=True).head(10)
bottom_10_pred

,Name,Actual_Global_Sales,Predicted_Global_Sales
1484,nil admirari no tenbin teito genwaku toukidan,0.02,0.045102
1560,luxpain jp sales,0.03,0.059029
401,hakuouki ds,0.07,0.069929
1338,toki no kizuna sekigahara kitan,0.04,0.073123
785,clannad mitsumi mamoru sakamichi de gekan,0.03,0.076186
1003,hakuouki zuisouroku portable,0.09,0.083866
649,coderealize sousei no himegimi,0.15,0.084738
1151,nobunaga no yabou ds,0.01,0.089058
1697,doki majo plus,0.02,0.091453
748,otometeki koi kakumei love revo portable,0.02,0.095494


In [101]:
# Displaying bottom 10 global sales out of the test set
bottom_10 = results_df.sort_values(by="Actual_Global_Sales", ascending=True).head(10)
bottom_10

,Name,Actual_Global_Sales,Predicted_Global_Sales
25,sangoku koi senki omoide gaeshi cs edition,0.01,0.194611
1038,thunder alley,0.01,0.580021
1039,satomi hakkenden hachi tamanoki,0.01,0.154836
1725,himawari pebble in the sky portable,0.01,0.268943
1731,monster rancher advance 2,0.01,0.723897
1734,legoland,0.01,2.701149
503,mario luigi paper jam mario kart 7 double pack,0.01,0.467891
1840,tengai makyo dai yon no mokushiroku,0.01,0.266293
425,pachitte chonmage tatsujin 16 pachinko hissats...,0.01,0.254967
1708,il2 sturmovik,0.01,0.334898


In [102]:
# calculating test MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predictions)
print("Test MAE:", mae)

Test MAE: 0.646840866502289


In [103]:
# Joining results_df and df_raw on Name to add publisher, genre, franchisetag back to results_df
results_df = results_df.merge(df_raw[['Name', 'Publisher', 'Genre', 'FranchiseTag', 'Year', 'Decade']], on='Name', how='left')
results_df.head()

,Name,Actual_Global_Sales,Predicted_Global_Sales,Publisher,Genre,FranchiseTag,Year,Decade
0,chronicles of mystery the secret tree of life,0.08,0.229535,city interactive,adventure,0,2011,2010
1,back at the barnyard slop bucket games,0.14,0.442490,thq,sports,0,2008,2000
2,earth defense force 2025,0.51,0.455672,d3publisher,shooter,0,2013,2010
3,scoobydoo unmasked,0.75,0.571154,thq,platform,0,2005,2000
4,mlb 10 the show,1.32,0.885334,sony computer entertainment,sports,0,2010,2010


In [106]:
# saving results_df to csv
results_df.to_csv("test_results.csv", index=False)

In [109]:
# Displaying rows with highest prediction errors
results_df['Absolute_Error'] = abs(results_df['Actual_Global_Sales'] - results_df['Predicted_Global_Sales'])
results_df.sort_values(by='Absolute_Error', ascending=False).head(10)

,Name,Actual_Global_Sales,Predicted_Global_Sales,Publisher,Genre,FranchiseTag,Year,Decade,Absolute_Error
743,new super mario bros wii,28.62,3.075672,nintendo,platform,0,2009,2000,25.544328
1852,pokemon redpokemon blue,31.37,9.170599,nintendo,roleplaying,0,1996,1990,22.199401
1395,call of duty world at war,15.87,1.626569,activision,shooter,0,2008,2000,14.243431
1674,animal crossing wild world,12.27,0.431428,nintendo,simulation,0,2005,2000,11.838572
1257,red dead redemption,12.94,1.311295,taketwo interactive,action,0,2010,2010,11.628705
1060,super smash bros for wii u and 3ds,12.47,1.979508,nintendo,fighting,0,2014,2010,10.490492
1867,call of duty modern warfare trilogy,0.04,10.420180,activision,shooter,0,2016,2010,10.380180
982,nintendogs,24.76,14.524964,nintendo,simulation,0,2005,2000,10.235036
1411,the last of us,10.54,0.404384,sony computer entertainment europe,action,0,2013,2010,10.135616
1108,grand theft auto 2,3.42,13.551766,taketwo interactive,action,1,1998,1990,10.131766


In [111]:
# Displaying rows with smallest prediction errors
results_df['Absolute_Error'] = abs(results_df['Actual_Global_Sales'] - results_df['Predicted_Global_Sales'])
results_df.sort_values(by='Absolute_Error', ascending=True).head(10)

,Name,Actual_Global_Sales,Predicted_Global_Sales,Publisher,Genre,FranchiseTag,Year,Decade,Absolute_Error
401,hakuouki ds,0.07,0.069929,idea factory,adventure,0,2010,2010,0.000071
386,kouchuu ouja mushi king greatest champion e no...,0.19,0.190234,sega,action,0,2005,2000,0.000234
847,imagine party babyz,0.39,0.390243,ubisoft,simulation,0,2008,2000,0.000243
460,tamagotchi no kirakira omisecchi,0.22,0.219462,namco bandai games,misc,0,2008,2000,0.000538
92,yugioh world championship 2008,0.28,0.280663,konami digital entertainment,strategy,0,2007,2000,0.000663
1817,shin megami tensei persona 3 fes,0.63,0.628325,tecmo koei,roleplaying,1,2007,2000,0.001675
646,the mummy tomb of the dragon emperor,0.43,0.428260,vivendi games,action,0,2008,2000,0.001740
1482,my secret world by imagine,0.20,0.202184,ubisoft,misc,0,2008,2000,0.002184
845,army men world war land sea air,0.30,0.303009,3do,action,0,2000,2000,0.003009
1941,allpro football 2k8,0.41,0.406552,taketwo interactive,sports,0,2007,2000,0.003448
